In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/D2NN.git

import sys
sys.path.append('/content/D2NN')

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import time
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from module.one_dim.layers import *
from module.one_dim.generators import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train_pattern = categorical_output(y_train, width=14*14, padding=0.2)
y_test_pattern = categorical_output(y_test, width=14*14, padding=-0.2)

In [3]:
#@title デフォルトのタイトル テキスト
wavelength = 1550.0e-9 #@param {type:"number"}
d = 0.5e-6 #@param {type:"number"}
n = 1.0 #@param {type:"number"}

In [4]:
def create_model(**kwargs):
    inputs = tf.keras.Input((28, 28))
    x = ImageResizing([14, 14])(inputs)
    x = ImageTo1D()(x)
    x = IntensityToElectricField()(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=40.0e-6, d=d, n=n, method="expand")(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=40.0e-6, d=d, n=n, method="expand")(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=40.0e-6, d=d, n=n, method="expand")(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=40.0e-6, d=d, n=n, method="expand")(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=40.0e-6, d=d, n=n, method="expand")(x)
    x = ElectricFieldToIntensity(normalization="max")(x)
    # x = Detector(10, padding=0.2)(x)
    model = tf.keras.Model(inputs, x)
    return model

In [6]:
model = create_model()
model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    #loss=tf.keras.losses.sparse_categorical_crossentropy,
    loss=tf.keras.losses.mean_squared_error,
    # metrics=['loss']
    )

epochs = 30
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0.05,
    patience=2,
)

model_name = "20220608_1"
cholab_path = "/content/drive/MyDrive/D2NN/"
checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

result = model.fit(x_train,
                   y_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_test, y_test),
                   callbacks=[cp_callback]
                   )

path = cholab_path + "trained_model/"+ model_name
model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 14, 14)           0         
 zing)                                                           
                                                                 
 image_to1d_1 (ImageTo1D)    (None, 196)               0         
                                                                 
 intensity_to_electric_field  (None, 196)              0         
 _1 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 modulation_5 (Modulation)   (None, 196)               197       
                                                           

2022-06-08 16:11:13.289813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - ETA: 0s - loss: 24.7798

2022-06-08 16:12:40.298434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 94s 99ms/step - loss: 24.7798 - val_loss: 24.3323
Epoch 2/30
938/938 [==============================] - 93s 99ms/step - loss: 24.2431 - val_loss: 24.0174
Epoch 3/30
938/938 [==============================] - 92s 98ms/step - loss: 23.9681 - val_loss: 23.7641
Epoch 4/30
938/938 [==============================] - 93s 99ms/step - loss: 23.7088 - val_loss: 23.5103
Epoch 5/30
938/938 [==============================] - 92s 99ms/step - loss: 23.4579 - val_loss: 23.2671
Epoch 6/30
938/938 [==============================] - 96s 102ms/step - loss: 23.2287 - val_loss: 23.0544
Epoch 7/30
938/938 [==============================] - 91s 97ms/step - loss: 23.0324 - val_loss: 22.8757
Epoch 8/30
938/938 [==============================] - 91s 97ms/step - loss: 22.8659 - val_loss: 22.7138
Epoch 9/30
938/938 [==============================] - 93s 99ms/step - loss: 22.7166 - val_loss: 22.5729
Epoch 10/30
938/938 [==============================] - 91s 97ms/step - los

KeyboardInterrupt: 

In [26]:
model = create_model()
model.save("trained_model/test")

INFO:tensorflow:Assets written to: trained_model/test/assets


In [9]:
fig = plt.figure(figsize=(6, 6))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
ax1.plot(range(1, epochs + 1), result.history['loss'], color='black', label='Loss')
ax1.set_ylabel('Loss')
ax1.set_xlabel('epoch')
ax2.plot(range(1, epochs + 1), result.history['accuracy'], color='red', label='Accuracy')
ax2.set_ylabel('Accuracy')
# label1と2には、凡例用に各labelのリスト情報が入る
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
# 凡例をまとめて出力する
ax1.legend(handler1 + handler2, label1 + label2, loc=2, borderaxespad=0.)

In [11]:
model.evaluate(x_test, y_test)

2022-06-04 08:37:49.790385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
